# <span style="color:#0b486b">SIT 112 - Data Science Concepts</span>

---
Lecturer: Sergiy Shelyag | sergiy.shelyag@deakin.edu.au<br />


School of Information Technology, <br />
Deakin University, VIC 3215, Australia.

---
## <span style="color:#0b486b">Practical Session 7: K-Means Clustering with Scikit-Learn</span>

**The purpose of this session is to teach you about:**

1. Scikit-Learn Package
2. K-Means Clustering

---
## <span style="color:#0b486b">1. Scikit-Learn</span>



[Scikit-Learn](http://github.com/scikit-learn/scikit-learn) is a Python package designed to give access to **well-known** machine learning algorithms within Python code, through a **clean, well-thought-out API**. It has been built by hundreds of contributors from around the world, and is used across industry and academia.

Scikit-Learn is built upon Python's [NumPy (Numerical Python)](http://numpy.org) and [SciPy (Scientific Python)](http://scipy.org) libraries, which enable efficient in-core numerical and scientific computation within Python. As such, scikit-learn is not specifically designed for extremely large datasets, though there is [some work](https://github.com/ogrisel/parallel_ml_tutorial) in this area.

### <span style="color:#0b486b">1.1 Representation of Data in Scikit-learn</span>

Machine learning is about creating models from data: for that reason, we'll start by
discussing how data can be represented in order to be understood by the computer.  Along
with this, we'll build on our matplotlib examples from the previous section and show some
examples of how to visualize data.

Most machine learning algorithms implemented in scikit-learn expect data to be stored in a
**two-dimensional array or matrix**.  The arrays can be
either ``numpy`` arrays, or in some cases ``scipy.sparse`` matrices.
The size of the array is expected to be `[n_samples, n_features]`

- **n_samples:**   The number of samples: each sample is an item to process (e.g. classify).
  A sample can be a document, a picture, a sound, a video, an astronomical object,
  a row in database or CSV file,
  or whatever you can describe with a fixed set of quantitative traits.
- **n_features:**  The number of features or distinct traits that can be used to describe each
  item in a quantitative manner.  Features are generally real-valued, but may be boolean or
  discrete-valued in some cases.

The number of features must be fixed in advance. However it can be very high dimensional
(e.g. millions of features) with most of them being zeros for a given sample. This is a case
where `scipy.sparse` matrices can be useful, in that they are
much more memory-efficient than numpy arrays.

Scikit-Learn package include several datasets that you can load and start playing with them. You can consult with the documentation for details of the provided datasets. For example the [iris dataset.](http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

In [ ]:
iris.keys()

In [ ]:
X = iris['data']
X.shape

---
## <span style="color:#0b486b">2 Introducing K-Means</span>

K Means is an algorithm for **unsupervised clustering**: that is, finding clusters in data based on the data attributes alone (not the labels).

K Means is a relatively easy-to-understand algorithm.  It searches for cluster centers which are the mean of the points within them, such that every point is closest to the cluster center it is assigned to.

Let's look at how KMeans operates on the simple clusters we looked at previously. To emphasize that this is unsupervised, we'll not plot the colors of the clusters at first as if we don't have that data:

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

# use seaborn plotting defaults
import seaborn as sns
sns.set()

In [ ]:
from sklearn.datasets import make_blobs

col = ['r','g','b','y']


n_centers = 4
X, y = make_blobs(n_samples=300, centers=n_centers,
                  random_state=0, cluster_std=0.60)
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], c = y, s=10)
ax.set_title("simulated data without cluster labels")


By eye, it is relatively easy to pick out the four clusters. If you were to perform an exhaustive search for the different segmentations of the data, however, the search space would be exponential in the number of points. Fortunately, there is a well-known *Expectation Maximization (EM)* procedure which scikit-learn implements, so that KMeans can be solved relatively quickly. But before that, lets visualize the distance matrix.

In [ ]:
def Euclidean_distance(x,y):
    '''
    Compute the Euclidean distance between two vectors x and y
    '''
    dist = (np.array(x) - np.array(y))*(np.array(x) - np.array(y))
    return np.sqrt(dist.sum())

In [ ]:
n_rows = X.shape[0]
print(n_rows)

X2 = []
for k in range(n_centers):
    X2.append(X[y == k])

X3 = np.array([])
X3 = np.append(X3, X2)
X3 = X3.reshape(300, 2)

In [ ]:
euclidean_distances = np.zeros((n_rows, n_rows))

In [ ]:
# compute the Euclidean distance matrix using the Euclidean_distance function()

for i in range(n_rows):
    print(i, end=' ')
    for j in range(n_rows):
        euclidean_distances[i, j] = Euclidean_distance(X3[i, :], X3[j, :])

In [ ]:
# Visualise this distance matrix

fig, ax = plt.subplots(figsize=(8, 8))
cax = ax.imshow(euclidean_distances)
cbar = fig.colorbar(cax)

Now clustering:

In [ ]:
from sklearn.cluster import KMeans

est = KMeans(n_clusters=4)
est.fit(X)

y_kmeans = est.predict(X)
# y_kmeans = est.labels_

fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50, cmap='rainbow');
ax.set_title('Estimated Clusters')

The algorithm identifies the four clusters of points in a manner very similar to what we would do by eye!

## <span style="color:#0b486b">2.1 The K-Means Algorithm: Expectation Maximization</span>


K-Means is an example of an algorithm which uses an *Expectation-Maximization* approach to arrive at the solution.
*Expectation-Maximization* is a two-step approach which works as follows:

0. Guess some cluster centers
0. Repeat until converged
  0. Assign points to the nearest cluster center
  0. Set the cluster centers to the mean
   
Let's quickly visualize this process. First execute the cell below to import `plot_kmeans_interactive()` which is needed for interactive visualization.

In [ ]:
import prac7_utils

In [ ]:
prac7_utils.plot_kmeans_interactive()

This algorithm will (often) converge to the optimal cluster centers.

---
## <span style="color:#0b486b">2.2 Review</span>


In [ ]:
# Simulate the data

n_centers = 4

X, y = make_blobs(n_samples=300, centers=n_centers,
                  random_state=0, cluster_std=0.60)
fig, ax = plt.subplots()
ax.scatter(X[:, 0], X[:, 1], s=50);

In [ ]:
est = KMeans(n_clusters=4, init='k-means++', n_init=1, max_iter=100, tol=0.0001, verbose=True)
est.fit(X)

In [ ]:
est.predict(X)

In [ ]:
est.labels_

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(15, 6))

ax[0].scatter(X[:, 0], X[:, 1], c=est.labels_, s=50, cmap='rainbow');
ax[0].set_title('True Clusters')

ax[1].scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='rainbow');
ax[1].set_title('True Clusters')

### <span style="color:#0b486b">Note</span>

The cluster labels are not necesarily same as the true labels. This is not important for us. We are only interested in finding the clusters. Remeber KMeans is an unsupervised algorithm. How would it know the cluster labels.

---
## <span style="color:#0b486b">2.4 KMeans for Digits Clustering</span>


For a closer-to-real-world example, let's use a KMean to cluser the digits data provided by Scikit-Learn.

Go to http://scikit-learn.org/stable/datasets/index.html#datasets
to choose one dataset (Section 5.2).

First load up the data:

In this specific example we can look at the data by simply plotting it. Why?

Look at some of the data to see the difference between people's handwriting.

Setup your KMeans estimator and fit the data:

Visualize the cluster centers and confirm that even though we did not have access to the true labels, the cluster centers are recognizable.

We mentioned earlier that cluster lables might permute. Investigate and fix it.

<small><i>Some this notebook's material is taken from [Jake Vanderplas](http://www.vanderplas.com) presentation in PyCon 2015.</i></small>